In [ ]:
# --------------------------
# Required Imports
# --------------------------
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import matplotlib.cm as cm
import numpy as np

# --------------------------
# Output widget for results
# --------------------------
output = widgets.Output()

# --------------------------
# Fetch data from API
# --------------------------
def fetch_movies(db_choice):
    """
    Fetch movies from Flask API.
    db_choice: 'redis' or 'postgres'
    """
    try:
        print(f"Sending request to API for db='{db_choice}'")
        response = requests.get(
            f"http://127.0.0.1:5000/movies?db={db_choice}",
            headers={"Cache-Control": "no-cache"}  # prevent caching
        )
        response.raise_for_status()
        result = response.json()
        print(f"API returned source: {result.get('source', 'No source')}")
        return result
    except Exception as e:
        print("API error:", e)
        return None

# --------------------------
# Plot charts
# --------------------------
def plot_charts(df, source_label):
    genres = df.groupby("genre")["box_office_million_USD"].sum().sort_values(ascending=False)
    genre_counts = df["genre"].value_counts().sort_values(ascending=False)
    
    # Generate a color palette
    n_colors = len(genres)
    colors = cm.tab20(np.linspace(0, 1, n_colors))
    
    # Bar chart: Box Office by Genre
    plt.figure(figsize=(10,5))
    genres.plot(kind='bar', color=colors)
    plt.title(f"Box Office by Genre ({source_label})")
    plt.xlabel("Genre")
    plt.ylabel("Box Office (Million USD)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Pie chart: Genre distribution
    plt.figure(figsize=(6,6))
    genre_counts.plot(kind='pie', autopct='%1.1f%%', colors=colors)
    plt.title(f"Genre Distribution ({source_label})")
    plt.ylabel("")
    plt.show()

# --------------------------
# Display function
# --------------------------
def display_movies(change=None):
    """
    Fetch and display movies + charts for selected database.
    Only updates the output widget.
    """
    with output:
        output.clear_output(wait=True)  # Clear previous results
        
        db_choice = db_dropdown.value
        print(f"Dropdown selected: {db_choice}")
        
        result = fetch_movies(db_choice)
        
        if not result:
            print("No result returned from API")
            return
        
        if "error" in result:
            print(f"API Error: {result['error']}")
            return
        
        if "data" not in result:
            print("API response missing 'data'")
            return
        
        df = pd.DataFrame(result["data"])
        source = result.get("source", "Unknown Source")
        
        print(f"Data Source: {source}")
        print(f"Number of Records: {result.get('count', len(df))}")
        print(f"Execution Time: {result.get('execution_time_ms', 'N/A')} ms")
        print(f"Cache Status: {'Hot Redis subset' if 'Redis' in source else 'Full PostgreSQL dataset'}")
        
        if not df.empty:
            display(df)
            plot_charts(df, source)

# --------------------------
# Dropdown Widget
# --------------------------
db_dropdown = widgets.Dropdown(
    options=["redis", "postgres"],
    value="redis",  # default to Redis (hot subset)
    description="Database:"
)

# Observe dropdown changes
db_dropdown.observe(display_movies, names='value')

# --------------------------
# Display UI
# --------------------------
display(db_dropdown, output)

# Initial display
display_movies()


interactive(children=(Dropdown(description='Database:', options=('redis', 'postgres'), value='redis'), Output(…

: 